In [31]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler

from psotorch_lorenzo import PSO


def load_tensors(dataloc, shuffle=True, validation_split=0.5, random_seed=12345, phi=lambda x: x):
    data = np.loadtxt(dataloc)

    dataset_size = len(data)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    train_data = torch.from_numpy(data[train_indices]).float()
    val_data = torch.from_numpy(data[val_indices]).float()

    return phi(train_data[:, :2]), torch.squeeze(train_data[:, 2:]), phi(val_data[:, :2]), torch.squeeze(val_data[:, 2:])


def phi(X):
    squaredX = torch.pow(X, 2)
    return torch.hstack([X, squaredX])


class SingleLayerPerceptron(nn.Module):

    def __init__(self):
        super(SingleLayerPerceptron, self).__init__()
        self.fc = nn.Linear(6, 2)


    def forward(self, x):
        x = self.fc(x)
        return x


class SpiralClassifier(nn.Module):

    def __init__(self):
        super(SpiralClassifier, self).__init__()
        self.fc1 = nn.Linear(4, 6)
        self.fc2 = nn.Linear(6, 1)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"

device = torch.device(dev)

np.random.seed(12345)

num_epochs = 500
x_train, y_train, x_test, y_test = load_tensors('data/two_spirals.dat', device, phi=phi)

x_train = x_train.to(device)
y_train = y_train.to(device)

# model = SpiralClassifier().to(device)
model = SpiralClassifier().to(device)
loss = nn.BCEWithLogitsLoss().to(device)

optimizer = PSO(x_train, y_train, model=model, loss=loss, dim=35, inertia=0.7, a1=2.02, a2=2.02, population_size=30, time_steps=1000, search_range=1)

for i in range(num_epochs):

    if torch.cuda.is_available():
        y_train_preds = model(x_train.cuda())
        fitness = loss(y_train_preds, y_train.type(torch.FloatTensor).cuda().reshape(132,1))
    else:
        y_train_preds = model(x_train)
        fitness = loss(y_train_preds[:, 0], y_train.type(torch.FloatTensor))
    
    class_classified = [1 if i > 0.5 else 0 for i in torch.sigmoid(y_train_preds) ]
    accuracy = sum([1 for (i,j) in list(zip(y_train.type(torch.FloatTensor).cuda().reshape(132,1),class_classified))if i==j])/y_train_preds.shape[0]

    print(f"Epoch {i}; Acc={accuracy}")

    optimizer.step()

Epoch 0; Acc=0.5378787878787878
Epoch 1; Acc=0.5378787878787878
Epoch 2; Acc=0.553030303030303
Epoch 3; Acc=0.4772727272727273
Epoch 4; Acc=0.4696969696969697
Epoch 5; Acc=0.44696969696969696
Epoch 6; Acc=0.5075757575757576
Epoch 7; Acc=0.5681818181818182
Epoch 8; Acc=0.5681818181818182
Epoch 9; Acc=0.49242424242424243
Epoch 10; Acc=0.5
Epoch 11; Acc=0.48484848484848486
Epoch 12; Acc=0.5833333333333334
Epoch 13; Acc=0.5303030303030303
Epoch 14; Acc=0.553030303030303
Epoch 15; Acc=0.5454545454545454
Epoch 16; Acc=0.5454545454545454
Epoch 17; Acc=0.5606060606060606
Epoch 18; Acc=0.5606060606060606
Epoch 19; Acc=0.553030303030303
Epoch 20; Acc=0.553030303030303
Epoch 21; Acc=0.5454545454545454
Epoch 22; Acc=0.5454545454545454
Epoch 23; Acc=0.5454545454545454
Epoch 24; Acc=0.553030303030303
Epoch 25; Acc=0.553030303030303
Epoch 26; Acc=0.553030303030303
Epoch 27; Acc=0.553030303030303
Epoch 28; Acc=0.553030303030303
Epoch 29; Acc=0.5757575757575758
Epoch 30; Acc=0.5681818181818182
Epoch 31

Epoch 250; Acc=0.5227272727272727
Epoch 251; Acc=0.5151515151515151
Epoch 252; Acc=0.5227272727272727
Epoch 253; Acc=0.5227272727272727
Epoch 254; Acc=0.5227272727272727
Epoch 255; Acc=0.5227272727272727
Epoch 256; Acc=0.5303030303030303
Epoch 257; Acc=0.5303030303030303
Epoch 258; Acc=0.5227272727272727
Epoch 259; Acc=0.5227272727272727
Epoch 260; Acc=0.5227272727272727
Epoch 261; Acc=0.5378787878787878
Epoch 262; Acc=0.553030303030303
Epoch 263; Acc=0.5454545454545454
Epoch 264; Acc=0.5454545454545454
Epoch 265; Acc=0.5378787878787878
Epoch 266; Acc=0.5303030303030303
Epoch 267; Acc=0.5303030303030303
Epoch 268; Acc=0.5227272727272727
Epoch 269; Acc=0.5227272727272727
Epoch 270; Acc=0.5303030303030303
Epoch 271; Acc=0.5303030303030303
Epoch 272; Acc=0.5378787878787878
Epoch 273; Acc=0.5303030303030303
Epoch 274; Acc=0.5378787878787878
Epoch 275; Acc=0.5378787878787878
Epoch 276; Acc=0.5378787878787878
Epoch 277; Acc=0.5378787878787878
Epoch 278; Acc=0.5378787878787878
Epoch 279; Acc=